In [1]:
import sys
import os
import numpy as np
import napari
%matplotlib inline
from matplotlib import pyplot as plt  # graphic library, for plots
import numba as nb
import tifffile
import skimage

In [2]:
import filedialogs

In [3]:
from metavision_core.event_io.raw_reader import RawReader
from metavision_core.event_io.py_reader import EventDatReader
from metavision_core.event_io import EventsIterator
from metavision_sdk_cv import ActivityNoiseFilterAlgorithm

### Loading Datasets

In [ ]:
#path = filedialogs.gui_fname('example_data/')
#path = path.decode('ascii')

In [4]:
path = 'C:/Users/B0067-WS-2/Documents/code/github-ffvoigt/accordion/playground/example_data/bloodflow_5s.raw'

In [5]:
record_raw = RawReader(path, max_events=100000000)
print(record_raw)

RawReader(C:/Users/B0067-WS-2/Documents/code/github-ffvoigt/accordion/playground/example_data/bloodflow_5s.raw)
current time : 0us done : False
current event index : 0
_begin_buffer 0,_end_buffer_ 0,  buffer_size 100000000


In [6]:
events = record_raw.load_n_events(90000000)
print('Start Time:  ', events['t'][0]/1000, ' ms')
print('End Time:    ', events['t'][-1]/1000, ' ms')
print('Total Events:', len(events))

Start Time:   6.96  ms
End Time:     4999.998  ms
Total Events: 76169784


### Noise filtering
For whatever reason: If `filtered_events_` and `filtered_events` are the same variable, the export doesn't work

In [7]:
print('Started with: ', len(events),' events' )
noisereductionfilter = ActivityNoiseFilterAlgorithm(1280,720,1000)
filtered_events_ = noisereductionfilter.get_empty_output_buffer()
noisereductionfilter.process_events(events, filtered_events_)
filtered_events = filtered_events_.numpy()
print('After filtering: ', len(filtered_events),' events' )

Started with:  76169784  events
After filtering:  57327737  events


### Quick visualization

In [ ]:
def viz_events(events, height, width):
    img = np.full((height, width, 3), 128, dtype=np.uint8)
    img[events['y'], events['x']] = 255 * events['p'][:, None]
    return img

In [ ]:
height, width = record_raw.get_size()
im = viz_events(events, height, width)
plt.imshow(im)
plt.tight_layout()

In [ ]:
height, width = record_raw.get_size()
im = viz_events(filtered_events, height, width)
plt.imshow(im)
plt.tight_layout()

In [ ]:
im_rotated = skimage.transform.rotate(im, -36)

In [ ]:
plt.imshow(im_rotated)
plt.tight_layout()

In [ ]:
im_rotated_crop = im_rotated[120:400]

In [ ]:
plt.imshow(im_rotated_crop)
plt.tight_layout()

### Conversion to image-like array
* 8bit, Z or T, X, Y, C 
* e.g. np.random.randint(0, 255, (100, 720, 1280, 3), 'uint8')

In [8]:
@nb.njit
def events_to_stack(events, dt=1000, imagesize=(720, 1280)):
    height, width = imagesize
    min_timepoint = events['t'].min()
    max_timepoint = events['t'].max()
    
    timepoints = int(np.ceil((max_timepoint - min_timepoint)/dt))    
    
    # generate a numpy array with the width 
    eventstack = np.zeros((timepoints,height,width,2), dtype='uint8')
    
    for event in events:
        x_pos = event[1]
        y_pos = event[0]
        timepoint = np.floor_divide((event[3]-min_timepoint),dt)
        if event[2] == 0:
            if eventstack[timepoint, x_pos, y_pos, 0] < 255:
                eventstack[timepoint, x_pos, y_pos, 0] += 1
        else: 
            if eventstack[timepoint, x_pos, y_pos, 1] < 255:
                eventstack[timepoint, x_pos, y_pos, 1] += 1
             
    
    return eventstack 

In [9]:
eventstack = events_to_stack(events, dt=1000)
print('Size of the eventstack in GB: ', eventstack.size/8/1024/1024/1024)

Size of the eventstack in GB:  1.0715961456298828


In [ ]:
eventstack.shape

In [10]:
filtered_eventstack = events_to_stack(filtered_events, dt=1000)
print('Size of the filtered eventstack in GB: ', eventstack.size/8/1024/1024/1024)

Size of the filtered eventstack in GB:  1.0715961456298828


### Rotate Dataset

In [ ]:
testim = filtered_eventstack[0]

In [ ]:
plt.imshow(testim[:,:,0])
plt.tight_layout()

In [ ]:
def rotate_stack(stack, angle):
    rotated_stack = np.zeros_like(stack)
    for plane in range(len(stack)):
        rotated_stack[plane,:,:,0] = skimage.transform.rotate(stack[plane,:,:,0], angle)
        rotated_stack[plane,:,:,1] = skimage.transform.rotate(stack[plane,:,:,1], angle)
    return rotated_stack

In [ ]:
import time

In [ ]:
rotated_stack_plane = skimage.transform.rotate(filtered_eventstack[300,:,:,0], -36)

In [ ]:
time1 = time.time()
filtered_events_rotated = rotate_stack(filtered_eventstack, -36)
time2 = time.time()
dt = time2-time1
print('Time in s: ', dt)

In [ ]:
filtered_eventstack_crop = filtered_events_rotated[:,120:400,:]

In [ ]:
testim2 = filtered_eventstack_crop[10]

In [ ]:
plt.imshow(rotated_stack_plane)
plt.tight_layout()

In [ ]:
filtered_eventstack_crop = filtered_events_rotated[:,120:400,:]

In [ ]:
filtered_eventstack_crop.shape

In [ ]:
np.max(filtered_eventstack_crop)

### Napari visualization

In [16]:
viewer = napari.Viewer()

toggle_ndisplay() missing 1 required positional argument: 'viewer'
Traceback (most recent call last):
  File "C:\Users\B0067-WS-2\.conda\envs\eventcam1\lib\site-packages\in_n_out\_store.py", line 805, in _exec
    result = func(**bound.arguments)
TypeError: toggle_ndisplay() missing 1 required positional argument: 'viewer'
roll_axes() missing 1 required positional argument: 'viewer'
Traceback (most recent call last):
  File "C:\Users\B0067-WS-2\.conda\envs\eventcam1\lib\site-packages\in_n_out\_store.py", line 805, in _exec
    result = func(**bound.arguments)
TypeError: roll_axes() missing 1 required positional argument: 'viewer'


In [ ]:
my_new_layer = viewer.add_image(filtered_eventstack, channel_axis=3, contrast_limits=[[0,2],[0,2]], name=['ON Events', 'Off Events'])

In [ ]:
my_new_layer = viewer.add_image(filtered_eventstack_crop, channel_axis=3, contrast_limits=[[0,2],[0,2]], name=['ON Events', 'Off Events'])

### Exporting data

In [12]:
on_events = filtered_eventstack[:,:,:,1]

In [13]:
off_events = filtered_eventstack[:,:,:,0]

In [15]:
tifffile.imwrite('bloodflow_5s_1ms_off_events.tif', off_events, photometric='minisblack')